In [1]:
import os

import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm

import torch
from torchvision.transforms import ToTensor
from facenet_pytorch import MTCNN, InceptionResnetV1

In [2]:
mtcnn = MTCNN()
resnet = InceptionResnetV1(pretrained='vggface2').eval()
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
device = 'cuda'

In [3]:
resnet.to(device)

InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

In [4]:
embeds = []
data_path = 'test_data/test_data/'
paths = os.listdir(data_path)

for img_path in tqdm(paths):
    img = Image.open(data_path + img_path)
    face = mtcnn(img)

    if face is not None:
        embed = resnet(face.unsqueeze(0).cuda()).cpu().numpy()
    else:
        embed = resnet(ToTensor()(img).unsqueeze(0).cuda()).cpu().numpy()

    embeds.append(embed)

100%|██████████| 79396/79396 [7:20:23<00:00,  4.93it/s]  


In [5]:
np.save('embeds', embeds)